In [1]:
import Environment_non_stationary
import Environment
import Shop
from mab.ucb_learner import *
from utilities import *

In [2]:
shop_ts = Shop.Shop()
shop_c = Shop.Shop()

env = Environment_non_stationary.Environment(shop_ts.prices1, shop_ts.prices2, shop_ts.discounts, n_phases=4)
s_env = Environment.Environment(shop_ts.prices1,shop_ts.prices2,shop_ts.discounts)



In [3]:
price_arms = [(i, j) for i in range(len(shop_ts.prices1)) for j in range(len(shop_ts.prices2))]
n_price_arms = len(price_arms)

matching_arms = [(i, j, k, l) for i in range(len(shop_ts.prices1)) for j in range(len(shop_ts.prices2)) for k in range(shop_ts.n_classes) for l in range(len(shop_ts.discounts))]
n_matching_arms = len(matching_arms)

##2-STEP ts

shop_ts.set_price_learner('TS', n_price_arms)
shop_ts.set_assignment_learner('TS', n_matching_arms)

## SLIDING WINDOW TS
days=365
frame_size = int(days / env.n_phases)
shop_ts.set_price_learner_sw('TS', n_price_arms, frame_size=frame_size, days=days)
shop_ts.set_assignment_learner_sw('TS', n_matching_arms, frame_size=frame_size, days=days)


In [7]:
###EXPERIMENT FOR STATIONARY ENVIRONMENT
##GET CLAIRVOYANT PRICE
generated_promo_ratios = np.array([0.4, 0.25, 0.2, 0.15])
max_ind_reward = max(env.prices1) + max(env.prices2)



expected_rewards = np.zeros((5,5)) # price1*price2
shop_c.set_expected_customers(np.array([25, 25, 25, 25]))
customers_mean = np.array([25, 25, 25, 25])

##### other way to calculate clairvoyant
cec = shop_c.get_customer_list()
cep = shop_c.get_promo_list(generated_promo_ratios)

cmt, cpa, cer, ers  = shop_c.get_clairvoyant_prices_and_matching(cec, cep, 5, 5)
cp1idx = price_arms[cpa][0]
cp2idx = price_arms[cpa][1]
cp1 = env.prices1[cp1idx]
cp2 = env.prices2[cp2idx]

cpf = shop_c.get_promo_fractions_from_tuples(cmt)

print(cpf)
print(cp1)
print(cp2)
print(cer)

(4, 4)
[[0.6 0.  0.4 0. ]
 [1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.4 0.6]]
240.0
300.0
34603.359650931714


In [5]:
days = 365


ts_rewards_per_class_per_day = []
ts_sw_rewards_per_class_per_day = []
c_rewards_per_class_per_day = []

customers_em = np.array([0, 0, 0, 0])
customers_ie = np.array([25, 25, 25, 25])

for day in range(days):
    if day == 0:
        shop_ts.set_expected_customers(customers_ie)
        
    else:
        customers_em = (customers_em*(day-1) + env.customers) / day
        shop_ts.set_expected_customers((customers_em + 0.99).astype(int))
    # select the price for 2 step learner with no sliding window
    chosen_price_tuple_idx_ts = shop_ts.price_learner.pull_arm()

    price1idx_ts = price_arms[chosen_price_tuple_idx_ts][0]
    price2idx_ts = price_arms[chosen_price_tuple_idx_ts][1]
    price1_ts = env.prices1[price_arms[chosen_price_tuple_idx_ts][0]]
    price2_ts = env.prices2[price_arms[chosen_price_tuple_idx_ts][1]]

    # perform matching and get promo fractions for 2 step learner with no sliding window 
    expected_customers_ts = shop_ts.get_customer_list()
    expected_promos_ts = shop_ts.get_promo_list(generated_promo_ratios)
    matched_tuples_ts = shop_ts.assignment_learner.pull_arm_matching(expected_customers_ts, expected_promos_ts, matching_arms, price1idx_ts, price2idx_ts)
    promo_fractions_ts = shop_ts.get_promo_fractions_from_tuples(matched_tuples_ts)

    # perform matching and get promo fractions for learner with sliding window
    price_arm_idx_ts_sw = shop_ts.price_learner_sw.pull_arm(day)

    price1idx_ts_sw = price_arms[price_arm_idx_ts_sw][0]
    price2idx_ts_sw = price_arms[price_arm_idx_ts_sw][1]
    price1_ts_sw = env.prices1[price_arms[price_arm_idx_ts_sw][0]]
    price2_ts_sw = env.prices2[price_arms[price_arm_idx_ts_sw][1]]

    expected_customers_ts_sw = shop_ts.get_customer_list()
    expected_promos_ts_sw = shop_ts.get_promo_list(generated_promo_ratios)
    matched_tuples_ts_sw = shop_ts.assignment_learner_sw.pull_arm_matching(expected_customers_ts_sw, expected_promos_ts_sw, matching_arms, day, price1idx_ts_sw, price2idx_ts_sw)
    promo_fractions_ts_sw = shop_ts.get_promo_fractions_from_tuples(matched_tuples_ts_sw)

    # simulate the day
    env.generate_next_day_customers()

    enum_customers = list(enumerate(env.customers))
    customers = np.concatenate([np.ones(c).astype(int)*p for p,c in enum_customers])
    np.random.shuffle(customers)

    revenue_ts = []
    revenue_ts_sw = []
    revenue_c = []

    ts_total_reward = 0
    ts_sw_total_reward = 0
    c_total_reward = 0

    c_count = 0

    for cust in customers:
        ts_ind_reward = 0

        
        selected_promo_ts = np.random.choice([0, 1, 2, 3], 1, p=promo_fractions_ts[cust])
        selected_promo_ts_sw = np.random.choice([0, 1, 2, 3], 1, p=promo_fractions_ts_sw[cust])
        selected_promo_c = np.random.choice([0, 1, 2, 3], 1, p=cpf[cust])

        

        ts_buy1 = env.round1(cust, price1_ts)
        ts_buy2 = 0
        if ts_buy1:
            ts_buy2 = env.round2(cust, selected_promo_ts, price2_ts)

        
        ts_sw_buy1 = env.round1(cust, price1_ts_sw, days)
        ts_sw_buy2 = 0
        if ts_sw_buy1:
            ts_sw_buy2 = env.round2(cust, selected_promo_ts_sw, price2_ts_sw, days)

        ts_ind_reward = ts_buy1*price1_ts + ts_buy2*price2_ts*(1-shop_ts.discounts[selected_promo_ts])
        ts_sw_ind_reward = ts_sw_buy1*price1_ts_sw + ts_sw_buy2*price2_ts_sw*(1-shop_ts.discounts[selected_promo_ts_sw])
                

        # c_buy1 = env.round1(cust, cp1)
        # c_buy2 = 0
        # if c_buy1:
        #     c_buy2 = env.round2(cust, selected_promo_c, cp2)
        
        ts_ind_reward = ts_buy1*price1_ts + ts_buy2*price2_ts*(1-shop_ts.discounts[selected_promo_ts])
        ts_sw_ind_reward = ts_sw_buy1*price1_ts_sw+ ts_sw_buy2*price2_ts_sw*(1-shop_ts.discounts[selected_promo_ts_sw])
        c_ind_reward = env.true_conv1[cust, cp1idx]*cp1 + env.true_conv1[cust, cp1idx]*env.true_conv2[selected_promo_c, cust, cp2idx]*cp2*(1-shop_ts.discounts[selected_promo_c])

        
        norm_ts_ind_reward = ts_ind_reward / max_ind_reward
        norm_ts_sw_reward = ts_sw_ind_reward / max_ind_reward

        # update 2-step ts
        
        
        shop_ts.price_learner.update(chosen_price_tuple_idx_ts, norm_ts_ind_reward)
        matching_arm_index_ts = matching_arms.index((price1idx_ts, price2idx_ts, cust, selected_promo_ts))
        shop_ts.assignment_learner.update(matching_arm_index_ts, norm_ts_ind_reward)

        shop_ts.price_learner_sw.update(price_arm_idx_ts_sw, norm_ts_sw_reward)
        matching_arm_index_ts_sw = matching_arms.index((price1idx_ts_sw, price2idx_ts_sw ,cust,selected_promo_ts_sw))
        # buradaki matching arm index sadece price ikilisini veriyor

        
        revenue_ts.append(ts_ind_reward)
        revenue_ts.append(ts_sw_ind_reward)
        revenue_c.append(c_ind_reward)

    ts_rewards_per_class_per_day.append(np.mean(revenue_ts))
    ts_sw_rewards_per_class_per_day.append(np.mean(revenue_ts))
    c_rewards_per_class_per_day.append(np.mean(revenue_c))

IndexError: index 4 is out of bounds for axis 1 with size 4